In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


# Import Libraries 

In [2]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

# Load Data

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
test_data  = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

In [4]:
# Add an indicator column
train_data['Dataset'] = 'train'
test_data['Dataset'] = 'test'

# Concatenate train and test
df = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

# 1. Exploratory Data Analysis (EDA)

## 1.1: Load and Understand the Dataset

In [5]:
train_data

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Dataset
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0,train
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0,train
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0,train
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0,train
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0,train
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0,train
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0,train
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0,train


In [6]:
train_data.columns

Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount', 'Dataset'],
      dtype='object')

In [7]:
test_data

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Dataset
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House,test
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment,test
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo,test
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House,test
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.0,Female,38782.0,Married,1.0,Bachelor's,NaN,14.498639,Rural,...,NaN,8.0,309.0,2.0,2021-07-09 15:21:39.184157,Average,Yes,Daily,Condo,test
799996,1999996,NaN,Female,73462.0,Single,0.0,Master's,NaN,8.145748,Rural,...,2.0,0.0,NaN,2.0,2023-03-28 15:21:39.250151,Good,No,Daily,Apartment,test
799997,1999997,26.0,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,NaN,10.0,NaN,6.0,2019-09-30 15:21:39.132191,Poor,No,Monthly,Apartment,test
799998,1999998,34.0,Female,45661.0,Single,3.0,Master's,NaN,15.937248,Urban,...,2.0,17.0,467.0,7.0,2022-05-09 15:21:39.253660,Average,No,Weekly,Condo,test


# 2: Data Preprocessing

## 2.1: Explore Missing Values

In [8]:
pd.set_option('display.max_columns', None)
df.head()  # Display the first few rows with all columns

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Dataset
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0,train
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0,train
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0,train
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0,train
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0,train


In [9]:
test_data.isnull().sum()

id                           0
Age                      12489
Gender                       0
Annual Income            29860
Marital Status           12336
Number of Dependents     73130
Education Level              0
Occupation              239125
Health Score             49449
Location                     0
Policy Type                  0
Previous Claims         242802
Vehicle Age                  3
Credit Score             91451
Insurance Duration           2
Policy Start Date            0
Customer Feedback        52276
Smoking Status               0
Exercise Frequency           0
Property Type                0
Dataset                      0
dtype: int64

In [10]:
missing_percentage = (df.isnull().sum() / len(df)) * 100
print(missing_percentage.sort_values(ascending=False))

Premium Amount          40.00000
Previous Claims         30.34155
Occupation              29.86000
Credit Score            11.46665
Number of Dependents     9.14010
Customer Feedback        6.50500
Health Score             6.17625
Annual Income            3.74045
Age                      1.55970
Marital Status           1.54325
Vehicle Age              0.00045
Insurance Duration       0.00015
Property Type            0.00000
Exercise Frequency       0.00000
Smoking Status           0.00000
id                       0.00000
Policy Start Date        0.00000
Policy Type              0.00000
Location                 0.00000
Education Level          0.00000
Gender                   0.00000
Dataset                  0.00000
dtype: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Age                   float64
 2   Gender                object 
 3   Annual Income         float64
 4   Marital Status        object 
 5   Number of Dependents  float64
 6   Education Level       object 
 7   Occupation            object 
 8   Health Score          float64
 9   Location              object 
 10  Policy Type           object 
 11  Previous Claims       float64
 12  Vehicle Age           float64
 13  Credit Score          float64
 14  Insurance Duration    float64
 15  Policy Start Date     object 
 16  Customer Feedback     object 
 17  Smoking Status        object 
 18  Exercise Frequency    object 
 19  Property Type         object 
 20  Premium Amount        float64
 21  Dataset               object 
dtypes: float64(9), int64(1), object(12)
memory

In [12]:
# Separate numerical and categorical columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Display results
print("Numerical Columns:")
print(numerical_columns)

print("\nCategorical Columns:")
print(categorical_columns)

Numerical Columns:
Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Premium Amount'],
      dtype='object')

Categorical Columns:
Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type', 'Dataset'],
      dtype='object')


In [13]:
# Handling missing values for numerical columns
for column in numerical_columns:
    # Replace missing values in numerical columns with the mean
    df[column].fillna(df[column].mean(), inplace=True)

# Handling missing values for categorical columns
for column in categorical_columns:
    # Replace missing values in categorical columns with 'no activity'
    df[column].fillna('unknown', inplace=True)

In [14]:
df.isnull().sum()

id                      0
Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Insurance Duration      0
Policy Start Date       0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
Dataset                 0
dtype: int64

## 2.2: Encoding

In [15]:
categorical_columns

Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type', 'Dataset'],
      dtype='object')

In [16]:
# Convert 'Policy Start Date' to datetime format
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')

# # Extract meaningful features
# df['Policy Start Year'] = df['Policy Start Date'].dt.year
# df['Policy Start Month'] = df['Policy Start Date'].dt.month
# df['Policy Start Day'] = df['Policy Start Date'].dt.day
# df['Policy Start DayOfWeek'] = df['Policy Start Date'].dt.dayofweek
# df['Policy Start IsWeekend'] = df['Policy Start Date'].dt.dayofweek >= 5

# Extract features
df['year'] = df['Policy Start Date'].dt.year
df['month'] = df['Policy Start Date'].dt.month
df['day'] = df['Policy Start Date'].dt.day
# df['hour'] = df['Policy Start Date'].dt.hour
df['dayofweek'] = df['Policy Start Date'].dt.dayofweek


# # Create cyclical features for hour
# df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
# df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))

# Drop the original datetime column
df.drop('Policy Start Date', axis=1, inplace=True)

In [17]:
# cat = ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
#        'Policy Type', 'Policy Start Date', 'Customer Feedback',
#        'Smoking Status', 'Exercise Frequency', 'Property Type']

In [18]:
cat = ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type','Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type']

In [19]:
# Convert categorical columns to integers (Label Encoding)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in cat:
    df[col] = label_encoder.fit_transform(df[col])  # Apply LabelEncoder to each column

## 2.3: Train-Test Split for Model Development

In [20]:
from sklearn.model_selection import train_test_split

# Split concatenated data back into train and test
train_data = df[df['Dataset'] == 'train'].drop(columns=['Dataset', 'id'])
test_data = df[df['Dataset'] == 'test'].drop(columns=['Dataset', 'Premium Amount'])


# categorical_columns = train_data.select_dtypes(include=['object']).columns

# Example: Assume 'target_column' is your target variable
target_column = 'Premium Amount'  # Replace with the actual target column name
X = train_data.drop(columns=[target_column])  # Features
y = train_data[target_column]  # Target

# # Reshape y to be 2D (required by TabNet)
# y = y.values.reshape(-1, 1)


# Split the data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Log transform the target variables to stabilize variance
y_train = np.log1p(y_train)  # Log transform training target
y_test = np.log1p(y_test)      # Log transform validation target


# Display the shapes of the resulting splits
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (960000, 22)
X_test shape: (240000, 22)
y_train shape: (960000,)
y_test shape: (240000,)


# 3. Model Selection and Implementation

## CatBoost

In [21]:
# import pandas as pd
# from catboost import CatBoostRegressor, Pool
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error



# # Initialize the CatBoost model
# model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, loss_function='RMSE', 
#         cat_features=['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
#        'Policy Type', 'Policy Start Date', 'Customer Feedback',
#        'Smoking Status', 'Exercise Frequency', 'Property Type'])

# # Train the model
# model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)


# # Make predictions
# y_pred = model.predict(X_test)

# # Evaluate the model
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print(f"RMSE: {rmse}")

In [22]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Initialize the CatBoost model
model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, loss_function='RMSE', 
        cat_features=['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Customer Feedback', 'year', 'month', 'day', 'dayofweek',
       'Smoking Status', 'Exercise Frequency', 'Property Type'])

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)


# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

0:	learn: 1.0905630	test: 1.0913594	best: 1.0913594 (0)	total: 2.03s	remaining: 33m 51s
100:	learn: 1.0494851	test: 1.0507161	best: 1.0507161 (100)	total: 1m 54s	remaining: 16m 56s
200:	learn: 1.0476830	test: 1.0492887	best: 1.0492871 (188)	total: 3m 47s	remaining: 15m 6s
300:	learn: 1.0469599	test: 1.0489363	best: 1.0489363 (300)	total: 5m 48s	remaining: 13m 29s
400:	learn: 1.0463390	test: 1.0487149	best: 1.0487132 (398)	total: 7m 56s	remaining: 11m 51s
500:	learn: 1.0457995	test: 1.0485582	best: 1.0485576 (499)	total: 10m 2s	remaining: 10m
600:	learn: 1.0451843	test: 1.0483159	best: 1.0483144 (599)	total: 12m 11s	remaining: 8m 5s
700:	learn: 1.0447072	test: 1.0482028	best: 1.0482028 (700)	total: 14m 14s	remaining: 6m 4s
800:	learn: 1.0441854	test: 1.0481386	best: 1.0481386 (800)	total: 16m 21s	remaining: 4m 3s
900:	learn: 1.0436705	test: 1.0480200	best: 1.0480197 (898)	total: 18m 27s	remaining: 2m 1s
999:	learn: 1.0431969	test: 1.0479526	best: 1.0479510 (994)	total: 20m 27s	remaining

# Submition 

In [23]:
# Preprocess test data
test_features = test_data.drop(columns=['id'], errors='ignore')  # Drop unnecessary columns

In [24]:
# Predict using the trained CatBoost model
test_data['Premium Amount'] = model.predict(test_features)

In [25]:
# Reverse log transformation if applied during training
test_data['Premium Amount'] = np.expm1(test_data['Premium Amount'])  # Use np.expm1 if np.log1p was used during training

# Clip predictions to avoid negative values (optional)
test_data['Premium Amount'] = test_data['Premium Amount'].clip(lower=0)

In [26]:
# Create submission file
submission = test_data[['id', 'Premium Amount']]  # Include 'id' and the predicted target column
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")

Submission file created: submission.csv
